In [1]:
library(dplyr)
library(keras)
library(tensorflow)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal,
    union



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal,
    union



In [ ]:
data <- readRDS("data/warehouse/test_lstm.rds")

# Convertir les past_stats en une matrice et diviser en train/test
X_data <- array(NA, dim = c(nrow(data), 3, length(data$past_stats[[1]])))
for (i in 1:nrow(data)) {
  X_data[i,,] <- as.matrix(unlist(data$past_stats[[i]]))
}

# Extraire les autres variables (age, draft_rank, height) et les répéter 3 fois pour chaque timestep
X_additional <- aperm(
  array(
    rep(array(unlist(data[, c("age", "draft_rank", "height")])), times = 3),
    dim = c(nrow(data), ncol(data[, c("age", "draft_rank", "height")]), 3)
  ),
  perm = c(1, 3, 2)
)

# Combiner les deux matrices en ajoutant les variables supplémentaires à la dimension des features
X <- array(c(X_data, X_additional), dim = c(dim(X_data)[1], dim(X_data)[2], dim(X_data)[3] + dim(X_additional)[3]))

# La variable cible : game_score
y <- as.matrix(data$game_score)

In [ ]:
# Train model ------------------------------------------------------------

input_layer <- layer_input(shape = c(3, ncol(X)))  # Déclare l'entrée explicitement

output_layer <- input_layer %>%
  layer_lstm(units = 50, return_sequences = FALSE) %>%
  layer_dense(units = 1)

model <- keras_model(inputs = input_layer, outputs = output_layer)

# Compilation du modèle
model %>% compile(
  loss = 'mean_squared_error',
  optimizer = 'adam'
)